# GENIE Learning Scenarios with Carbon Taxes

**Pre-requisites**
- You have the *MESSAGEix* framework installed and working
- You have run Westeros baseline scenario (``westeros_baseline.ipynb``) and solved it successfully

In [1]:
import ixmp
import message_ix
import numpy as np
import pandas as pd
import yaml

from collections.abc import Mapping
from itertools import repeat
from message_ix.models import MESSAGE_ITEMS
from message_ix.utils import make_df

from message_ix.tools.add_learning import add_learning


%matplotlib inline

<IPython.core.display.Javascript object>

In [2]:
mp = ixmp.Platform("local", jvmargs=['-Xmx8G'])

## Making a clone of the existing scenario 'baseline'

Dump scenario to excel nad create a scenario from excel

In [3]:
base = message_ix.Scenario(
    mp, model="GENIE_sandbox", scenario="learning",
    )

Clone baseline scenario and add learning parameters

## Scenario generation

Here, we run the model under a range of CO2 tax scenaros

In [4]:
# Firstly, let's create an empty list to contain the emissions data
tax_growth = {'n':0,'l':0.015,'m':0.030,'h':0.045,'vh':0.060} #{'h':0.050} 
rnd_target = {'n':0,'l':0.5,'m':5,'h':50,'vh':500} #{'h':0.050} 

data_collect = {par: [] for par in list(tax_growth.keys())}


In [5]:
# Scenario setup
lrn = 'y'
#rnd = 'l'
lrn_val = 1 if lrn == 'y' else 0

In [6]:
for txs in tax_growth.keys():
    for rnd in rnd_target.keys():
        # define base number for emissions taxes in 2020
        Base2020 = 0 if txs == 'n' else 10 # $/tCO2

        # clone an editable scenario from the baseline
        scen = base.clone(
        "GENIE",
        "learning"+lrn+"_CT"+txs+"_RND"+rnd,
        txs+" carbon tax with"+rnd+"RND target",
        keep_solution=False,
        )
        scen.check_out()

        # add R&D target parameter
        rnd_target_df = make_df(
                "relation_lower",
                relation='dac_cap_global',
                node_rel='R11_GLB',
                year_rel= 2035,
                value=rnd_target[rnd],
                unit='-',
                )
        scen.add_par("relation_lower", rnd_target_df)

        # generate timeseries value of emissions taxes
        all_years = scen.set('year')
        all_nodes = scen.set('node')
        type_years = list(all_years[all_years>=2020])
        nodes = list(all_nodes[all_nodes=="World"])

        growth = tax_growth.get(txs)

        value_taxes = [round((Base2020*((1+growth)**(x-2020))),2) for x in type_years]

        # Generate emission tax data and add to the scenario
        mp.add_unit("USD/tCO2")
        tax_emission = []
        for n in nodes:
            data = make_df(
                "tax_emission",
                node=n,
                type_year=type_years,
                type_tec="all",
                unit="USD/tCO2",
                type_emission="TCE",
                value=value_taxes,
                )
            tax_emission.append(data)

        tax_emission = pd.concat(tax_emission)

        scen.add_par("tax_emission", tax_emission)

        # commit and solve the scenario    
        scen.commit(comment="GENIE with"+lrn+"learning |"+txs+"carbon tax |"+rnd+"RND target")

        scen.solve(gams_args =["--learningmode="+str(lrn_val)],solve_options={"threads":4,"scaind":-1})

        scen.var("OBJ")["lvl"]

        # List of technologies in electricity sector and daccs
        etechs = ['nuc_hc','nuc_lc','nuc_fbr',
                  'bio_istig','bio_istig_ccs','bio_ppl','h2_bio','h2_bio_ccs',
                  'eth_bio','eth_bio_ccs','liq_bio','liq_bio_ccs',
                  'coal_adv','coal_adv_ccs','coal_ppl','coal_ppl_u','igcc','igcc_ccs',
                  'foil_ppl','loil_cc','lio_ppl','oil_ppl','oil_ppl',
                  'gas_cc','gas_cc_ccs','gas_ct','gas_htfc','gas_ppl',
                  'geo_ppl',
                  'hydro_hc','hydro_lc',
                  'solar_pv_ppl','csp_sm1_ppl','csp_sm3_ppl','solar_th_ppl',
                  'wind_ppl','wind_ppf',
                  'SO2_scrub_ppl',
                  'dac_1_exports','dac_2_exports','dac_3_exports','dac_4_exports','dac_5_exports']
        years = [yr for yr in scen.set('year') if yr >= 2025]
        capacity = []

        for y in years:
            capacity.append(scen.var("CAP", filters={'year_act':y,'technology':etechs})
                            .groupby('technology').sum(numeric_only=True).reindex(etechs).fillna(0)
                            .drop(columns=['year_vtg','year_act','mrg'])
                            .rename(columns={'lvl':y})
                           )
        capacity = pd.concat(capacity, axis=1)

        with pd.ExcelWriter('output.xlsx', mode='a', engine='openpyxl', if_sheet_exists='overlay') as writer:  
            capacity.to_excel(writer, sheet_name="CT"+txs+"_RND"+rnd)


ModelError: GAMS errored with return code 1:
    Solver is to be called, the system should never return this number

For details, see the terminal output above, plus:
Listing   : C:\Users\pratama\Documents\GitHub\MESSAGEix\message_ix\message_ix\model\MESSAGE_run.lst
Input data: C:\Users\pratama\Documents\GitHub\MESSAGEix\message_ix\message_ix\model\data\MsgData_GENIE_learningy_CTn_RNDvh.gdx

## Close the connection to the database

In [ ]:
mp.close_db()